####Import any packages needed

In [1]:
import xml.etree.cElementTree as ET #Parse XML
from pprint import pprint #Pretty printing
import re #Regex
from collections import Counter #Count elements in list - good one!
from collections import defaultdict
from langdetect import detect #Language detection. Awesome. Returns most probable.
from langdetect import detect_langs #Probabilities of detected language. Seriously awesome.
import codecs
import json
from pymongo import MongoClient
import itertools
import math
import plotly.plotly as py
from plotly.graph_objs import *

#Data pre-processing

### Explore OSM Hampshire before importing to MongoDB

In [2]:
#Function that counts top-level tags as per Lesson 6 problem
def count_tags(filename):
    d={}
    with open(filename, "r", encoding="utf8") as osm_file:
        for _, elem in ET.iterparse(osm_file, events=("start",)):
            if elem.tag in d:
                d[elem.tag]+=1
            else:
                d[elem.tag]=1
    return d

In [5]:
top_lev_tags = count_tags("hampshire-latest.osm")

In [6]:
top_lev_tags

{'bounds': 1,
 'member': 71845,
 'nd': 2409885,
 'node': 1991437,
 'osm': 1,
 'relation': 3671,
 'tag': 1023892,
 'way': 261248}

####Define and find problematic patterns for street type as per Lesson 6

In [5]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [7]:
def key_type(element, keys):
    oth1 = 0
    if element.tag == "tag":
        if lower.search(element.attrib["v"]):
            keys["lower"]+=1
        else:
            oth1 += 1
        if lower_colon.search(element.attrib["v"]):
            keys["lower_colon"]+=1
        else:
            oth1 +=1
        if problemchars.search(element.attrib["v"]):
            keys["problemchars"]+=1
        else:
            oth1+=1
        if oth1==3:
            keys["other"]+=1
    return keys


def get_str_prob(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

In [9]:
str_type_prob = get_str_prob("hampshire-latest.osm")

In [10]:
str_type_prob

{'lower': 565037, 'lower_colon': 83, 'other': 233189, 'problemchars': 225583}

####Auditing street naming as per Lesson 6

In [61]:
#Define pattern for street specifier
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

#Expected street specifiers. Fairly generic, probably many more will be present in data set.
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Park", "Grove", "Close", "Crescent", "Gardens", "Way", "Mews","Walk", "Terrace"]

#Function that returns unexpected street types
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

#Function to quickly query street name tag            
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

#Function  for auditing the OSM XML dataset
def audit(osmfile):
    with open(osmfile, "r", encoding="utf8") as osm_file:
        street_types = defaultdict(set)
        for event, elem in ET.iterparse(osm_file, events=("start",)):
            if elem.tag == "node" or elem.tag == "way":
                for tag in elem.iter("tag"):
                    if is_street_name(tag):
                        audit_street_type(street_types, tag.attrib['v'])
    return street_types

####Run function on this OSM dataset

In [62]:
street_types = audit("hampshire-latest.osm")

In [64]:
pprint(street_types)

{'0ZS': {'SO51 0ZS'},
 '1EG': {'GU32 1EG'},
 '1NB': {'RG29 1NB'},
 '9BA': {'SO23 9BA'},
 'Acre': {'North Acre'},
 'Airport': {'Farnborough Airport', 'Blackbushe Airport'},
 'Alley': {'Bowling Alley'},
 'Alresford': {'Old Alresford'},
 'Approach': {'Station Approach'},
 'Ave': {'Franklin Ave'},
 'BERRYLANDS': {'BERRYLANDS'},
 'Beechcroft': {'Beechcroft'},
 'Berrylands': {'Berrylands'},
 'Boardwalk': {'The Boardwalk'},
 'Bottom': {'Owslebury Bottom'},
 'Bradley': {'Bradley'},
 'Bridge': {'Horseshoe Bridge'},
 'Broadacres': {'Broadacres'},
 'Broadcut': {'Broadcut'},
 'Broadway': {'Abbey Mill Gardens, The Broadway',
              'Midanbury Broadway',
              'The Broadway'},
 'Buildings': {'Hanover Buildings', 'Highbury Buildings', 'Gordon Buildings'},
 'Burkham': {'Burkham'},
 'Bury': {'The Bury'},
 'Cedars': {'The Cedars'},
 'Centre': {'Brighton Hill Centre',
            'The Chineham Centre',
            'The Marlands Shopping Centre',
            'The Swan Centre',
            '

Observations: I did not expect specifiers like "Parade"; but these are valid. Regarding problems, we can see that some are postcodes. Others are names of places rather than addresses. We have "Rd", "Raod", "Road--", "Rpad", "S", "STREET", "VIEW" full entry in caps, "X" which is probably unkown and best deleted, "lane", "road", "street"

Based on the above observations, I  made a mapper dictionary which will be used for replacing problematic types

In [65]:
mapping = {"Rd": "Road", "Raod": "Road", "Road--" :"Road", "Rpad": "Road", "S": "Street", "STREET": "Street", "VIEW": "View", "X": "-", "lane": "Lane", "road": "Road", "street": "Street"}

In [7]:
mapping

{'Raod': 'Road',
 'Rd': 'Road',
 'Road--': 'Road',
 'Rpad': 'Road',
 'S': 'Street',
 'STREET': 'Street',
 'VIEW': 'View',
 'X': '-',
 'lane': 'Lane',
 'road': 'Road',
 'street': 'Street'}

####Fixing street names

In [66]:
#This function aims to fix problems with street naming identified above
#Also included is a regex for UK postcodes
def update_name(name, mapping):
    postc = re.compile(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b')
    if postc.search(name):
        name = "-"
    elif name=="Junction of Drayton Lane Nr No. 159 Havant Road Drayton Portsmouth":
        name="Havant Road"
    elif name == "Solent Business Park, Whiteley, Hampshire":
        name = "Solent Business Park"
    elif name =="Junction St Ronan's Road outside school":
        name = "St.Ronan's Road"
    elif name=="S":
        name = "-"
    else:
        name1 = name.split()
        name2 = []
        for i in name1:
            if i in mapping.keys():
                i = mapping[i]
            if i!=None:
                name2.append(i.lower().title())
        name = " ".join(name2)
    return name

Sanity checking...

In [69]:
print("name", "| | | ", "new name")
for st_type, ways in street_types.items():
    for name in ways:
        better_name = update_name(name, mapping)
        if name != better_name:
            print(name, "| | | ", better_name)

name | | |  new name
Bluebell Raod | | |  Bluebell Road
SO51 0ZS | | |  -
THE STREET | | |  The Street
Reading Rpad | | |  Reading Road
Weir Road-- | | |  Weir Road
GU32 1EG | | |  -
Solent Business Park, Whiteley, Hampshire | | |  Solent Business Park
STABLE VIEW | | |  Stable View
Captain's Row | | |  Captain'S Row
Upper Arundel street | | |  Upper Arundel Street
BERRYLANDS | | |  Berrylands
X | | |  -
bluebell road | | |  Bluebell Road
Alton road | | |  Alton Road
Bluebell road | | |  Bluebell Road
OLDCORNE HOLLOW | | |  Oldcorne Hollow
Western Esplanade (corner of Fitzhugh Street) | | |  Western Esplanade (Corner Of Fitzhugh Street)
Junction St Ronan's Road outside school | | |  St.Ronan's Road
S | | |  -
Fair Oak Rd | | |  Fair Oak Road
Lower Farnham Rd | | |  Lower Farnham Road
Hythe Rd | | |  Hythe Road
Junction of Drayton Lane Nr No. 159 Havant Road Drayton Portsmouth | | |  Havant Road
 Saint John's North | | |  Saint John'S North
Saint John's North | | |  Saint John'S North
O

In reality not many values were "corrected". It also appears that the rules for fixing the case of the entries were not always adequate (as King's became King'S for examples)

The regex for UK postcodes was found here:
http://stackoverflow.com/questions/378157/python-regular-expression-postcode-search

In [9]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
tag_subel = ["amenity", "cuisine", "name", "phone"]

#Function that shapes the data according to the conceived schema prior to MongoDB entry
def shape_element(element):
    if element.tag == "node" or element.tag == "way" :
        #Initialise dictionary keys for subsequent completion
        d={}
        d["created"]={}
        d["pos"]=[]
        d["address"]={}
        d["amenity"]={}
        d["cuisine"]={}
        d["name"]={}
        d["phone"]={}
        d["type"]=element.tag
        for k,v in element.attrib.items():
            #Filling values for "created" key with relevant sub-dictionaries
            if k in CREATED:
                d["created"][k]=v
            #Filling latitude and longitude values for "pos" key
            elif k=="lat" or k=="lon":
                 if "pos" in d.keys():
                    d["pos"].append(float(v))
                 else:
                     d["pos"]=[float(v)]
            #Filling all others in their own key
            else:
                d[k]=v
            for subel in element:
                if subel.tag=="tag":
                    key = subel.attrib["k"]
                    value = subel.attrib["v"]
                    key_split=key.split(":")
                    if len(key_split)==2 and key_split[0]=="addr":
                        d["address"][key_split[1]]=value
                    elif key in tag_subel:
                        d[key] = value
        #Cleaning up for empty "pos" keys and fixing latitude,longitude order 
        if len(d["pos"])==0:
            del d["pos"]
        else:
            d["pos"][0], d["pos"][1] = d["pos"][1], d["pos"][0]
        #Cleaning up for empty "address"
        if d["address"]=={}:
            del d["address"]
        #General clean-up
        for key in tag_subel:
            if d[key]=={}:
                del d[key]
        #Fill-out values for "node_refs" key
        d["node_refs"]=[]
        if element.tag=="way":
            for subel in element:
                if subel.tag=="nd":
                    d["node_refs"].append(subel.attrib["ref"])
        if len(d["node_refs"])==0:
            del d["node_refs"]
        return d
    else:
        return None


def process_map(file_in, mapping, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            if element.tag == "node" or element.tag == "way":
                for tag in element.iter("tag"):
                    if is_street_name(tag):
                        tag.attrib['v'] = update_name(tag.attrib['v'], mapping)
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [ ]:
data_json = process_map("hampshire-latest.osm", mapping)

#MongoDB

###Insert into MongoDB

In [3]:
client = MongoClient("mongodb://localhost:27017")
db = client.osm
def insert_data(data, db):
    db.osm.insert(data)
    return

In [5]:
#db.osm.remove()
with open('hampshire-latest.osm.json') as f:
    for line in f:
        data=(json.loads(line))
        insert_data(data, db)

Briefly exploring data

In [148]:
pipe1 = {"$group":{"_id": "$address.city", "count":{"$sum": 1}}}
pipe2 = {"$sort": {"count": -1}}
pipeline = [pipe1, pipe2]

In [149]:
test = db.osm.aggregate(pipeline)
for i in test:
    pprint(i)

{'_id': None, 'count': 2233093}
{'_id': 'Southampton', 'count': 9487}
{'_id': 'Eastleigh', 'count': 6649}
{'_id': 'Winchester', 'count': 1268}
{'_id': 'Romsey', 'count': 625}
{'_id': "Chandler's Ford", 'count': 228}
{'_id': 'Liss', 'count': 171}
{'_id': 'Basingstoke', 'count': 165}
{'_id': 'Whitehill', 'count': 143}
{'_id': 'Denmead, Waterlooville', 'count': 119}
{'_id': 'Lymington', 'count': 70}
{'_id': 'Bordon', 'count': 64}
{'_id': 'Alton', 'count': 62}
{'_id': 'Portsmouth', 'count': 47}
{'_id': 'Andover', 'count': 32}
{'_id': 'Southsea', 'count': 27}
{'_id': 'Alresford', 'count': 24}
{'_id': 'Ropley', 'count': 22}
{'_id': 'Petersfield', 'count': 22}
{'_id': 'Fareham', 'count': 20}
{'_id': 'HASLEMERE', 'count': 19}
{'_id': 'Fordingbridge', 'count': 14}
{'_id': 'East Tisted', 'count': 14}
{'_id': 'Tadley', 'count': 14}
{'_id': 'Selborne', 'count': 12}
{'_id': 'Farnborough', 'count': 12}
{'_id': 'Lyndhurst', 'count': 12}
{'_id': 'Colden Common', 'count': 11}
{'_id': 'Aldershot', 'coun

Many city values missing... 

###List MongoDB field completion

In [6]:
MongoDB_entries = []
MongoDB_entries.append(db.osm.find().count())

In [6]:
element_list = ["amenity", "address", "address.city", "address.country", "address.place", "address.postcode", 
               "address.street", "address.housenumber", "address.interpolation", "address.district", "address.site", "type", "id", "name", "node_refs", "pos", "phone", "created", "created.changeset", "created.timestamp",
               "created.uid", "created.user", "created.version", "cuisine"]

In [7]:
MongoDB_element_dict = {}
for entry in element_list:
    MongoDB_element_dict[entry]=[db.osm.find({entry: {"$exists":1}}).count(), round(100*db.osm.find({entry: {"$exists":1}}).count()/MongoDB_entries[0], 2)]

Below we can see completion for each specified field

In [9]:
MongoDB_element_dict

{'address': [32704, 1.45],
 'address.city': [19592, 0.87],
 'address.country': [17397, 0.77],
 'address.district': [7489, 0.33],
 'address.housenumber': [26718, 1.19],
 'address.interpolation': [237, 0.01],
 'address.place': [1362, 0.06],
 'address.postcode': [18978, 0.84],
 'address.site': [48, 0.0],
 'address.street': [29613, 1.31],
 'amenity': [15701, 0.7],
 'created': [2252685, 100.0],
 'created.changeset': [2252685, 100.0],
 'created.timestamp': [2252685, 100.0],
 'created.uid': [2252667, 100.0],
 'created.user': [2252667, 100.0],
 'created.version': [2252685, 100.0],
 'cuisine': [794, 0.04],
 'id': [2252685, 100.0],
 'name': [80957, 3.59],
 'node_refs': [261248, 11.6],
 'phone': [401, 0.02],
 'pos': [1991437, 88.4],
 'type': [2252685, 100.0]}

An dhere more specifically within the address field

In [29]:
address_compl = {}
for name in element_list:
    a = name.split(".")
    if a[0]=="address":
        address_compl[name] = round(100*MongoDB_element_dict[name][0]/32704,2)

In [30]:
address_compl

{'address': 100.0,
 'address.city': 59.91,
 'address.country': 53.2,
 'address.district': 22.9,
 'address.housenumber': 81.7,
 'address.interpolation': 0.72,
 'address.place': 4.16,
 'address.postcode': 58.03,
 'address.site': 0.15,
 'address.street': 90.55}

### Exploring postcodes

In [85]:
pipe1 = {"$match": {"$and": [{"address": {"$exists": 1}}, {"address.postcode": {"$exists": 1}}]}}
#pipe2 = {"$group":{"_id": "$address.city", "count":{"$sum": 1}}}
pipe2 = {"$project": {"name": "$address.postcode"}}
pipeline=[pipe1, pipe2]
test = db.osm.aggregate(pipeline)

In [86]:
postc = re.compile(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b')
for i in test:
    if not postc.search(i['name']):
        pprint(i['name'])

'SO 17 9 JP'
'PO63EZ'
'PO7'
'SO21 EP'
'SO21 EP'
'SO21 EP'
'SO21 EP'
'SO21 EP'
'SO21'
'SO21'
'SO21'
'SO21'
'SO21 EP'
'SO21 1'
'SO50 4LI'
'SO50 4LI'
'SO53 1'
'SO53 2'
'Gu14 8TL'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FZ'
'SO172FW'
'SO172FW'
'PO13 OSG'
'SO172FW'
'SO172FW'
'GU14 OBW'
'SO17 3RI'
'SO17 3RI'
'SO17 3RI'
'SO17 3RI'
'SO17 3RI'
'SO17 3RI'
'SO17 3RI'
'SO17 3RI'
'PO1'
'GU46'
'SO53 1'
'SO53 1'
'SO53 1'
'SO53 1'
'SO53 2'
'SO53 2'
'SO53 2'
'SO206JF'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'SO53'
'PO3 6N'
'GU467UW'
'GU467UW'
'RG29'
'SO51'
'SO51'
'SO51'
'SO51'
'SO51'
'SO51'
'SO51'
'SO51'
'SO51'
'SO51'
'SO51'
'SO53'
'SO51'
'SO51'
'SO51'
'SO51'
'SO50 4LI'
'SO50 4LI'
'SO51 9 BY'
'SO51 9 BY'
'SO16 7LI'
'SO182QS'
'GU17  0NW'
'sp10 1ne'
' SO51 ONB'
'High Street'
'

The following issues are identified

    missing digits
    lower case
    no space
    O instead of 0
    stupid spaces
    typos
    street names instead of postcodes

###Exploring cities

In [105]:
pipe1 = {"$match": {"$and": [{"address": {"$exists": 1}}, {"address.city": {"$exists": 1}}]}}
pipe2 = {"$group":{"_id": "$address.city", "count":{"$sum": 1}}}
pipe4 = {"$project": {"name": "$address.city", "count":"$count"}}
pipe3 = {"$sort": {"count":-1}}
pipeline=[pipe1, pipe2, pipe3, pipe3]
test = db.osm.aggregate(pipeline)

In [106]:
for i in test:
        pprint(i)

{'_id': 'Southampton', 'count': 9487}
{'_id': 'Eastleigh', 'count': 6649}
{'_id': 'Winchester', 'count': 1268}
{'_id': 'Romsey', 'count': 625}
{'_id': "Chandler's Ford", 'count': 228}
{'_id': 'Liss', 'count': 171}
{'_id': 'Basingstoke', 'count': 165}
{'_id': 'Whitehill', 'count': 143}
{'_id': 'Denmead, Waterlooville', 'count': 119}
{'_id': 'Lymington', 'count': 70}
{'_id': 'Bordon', 'count': 64}
{'_id': 'Alton', 'count': 62}
{'_id': 'Portsmouth', 'count': 47}
{'_id': 'Andover', 'count': 32}
{'_id': 'Southsea', 'count': 27}
{'_id': 'Alresford', 'count': 24}
{'_id': 'Ropley', 'count': 22}
{'_id': 'Petersfield', 'count': 22}
{'_id': 'Fareham', 'count': 20}
{'_id': 'HASLEMERE', 'count': 19}
{'_id': 'Fordingbridge', 'count': 14}
{'_id': 'East Tisted', 'count': 14}
{'_id': 'Tadley', 'count': 14}
{'_id': 'Selborne', 'count': 12}
{'_id': 'Farnborough', 'count': 12}
{'_id': 'Lyndhurst', 'count': 12}
{'_id': 'Colden Common', 'count': 11}
{'_id': 'Aldershot', 'count': 11}
{'_id': 'Fleet', 'count'

No problems - need to do strip(), fix case and district in there for some however

###Exploring country

In [111]:
pipe1 = {"$match": {"$and": [{"address": {"$exists": 1}}, {"address.country": {"$exists": 1}}]}}
pipe2 = {"$group":{"_id": "$address.country", "count":{"$sum": 1}}}
pipe3 = {"$project": {"name": "$address.country", "count":"$count"}}
pipeline=[pipe1, pipe2]
test = db.osm.aggregate(pipeline)

In [112]:
for i in test:
        pprint(i)

{'_id': 'GB', 'count': 17397}


###Exploring name

In [123]:
pipe1 = {"$match": {"$and": [{"name": {"$exists": 1}}]}}
pipe2 = {"$group":{"_id": "$name", "count":{"$sum": 1}}}
pipe3 = {"$sort": {"count":-1}}
pipe4 = {"$project": {"name": "$name", "count":"$count"}}
pipeline=[pipe1, pipe2, pipe3, pipe4]
test = db.osm.aggregate(pipeline)

In [124]:
for i in test:
        pprint(i)

{'_id': 'South Western Main Line', 'count': 306}
{'_id': 'London Road', 'count': 241}
{'_id': 'Winchester Road', 'count': 241}
{'_id': 'High Street', 'count': 183}
{'_id': 'Church Lane', 'count': 169}
{'_id': 'Southampton Road', 'count': 151}
{'_id': 'Station Road', 'count': 149}
{'_id': 'South Coast Motorway', 'count': 133}
{'_id': 'New Road', 'count': 129}
{'_id': 'West Coastway Line', 'count': 127}
{'_id': 'Portsmouth Direct Line', 'count': 113}
{'_id': 'Wayfarers Walk', 'count': 109}
{'_id': 'Green Lane', 'count': 104}
{'_id': 'Quilter Road', 'count': 98}
{'_id': 'Mill Lane', 'count': 91}
{'_id': 'Botley Road', 'count': 89}
{'_id': 'The Avenue', 'count': 88}
{'_id': 'Romsey Road', 'count': 87}
{'_id': 'Farnborough Road', 'count': 84}
{'_id': 'Salisbury Road', 'count': 79}
{'_id': 'Petersfield Road', 'count': 75}
{'_id': 'Portsmouth Road', 'count': 72}
{'_id': 'Reading Road', 'count': 71}
{'_id': 'Church Road', 'count': 71}
{'_id': 'Grange Road', 'count': 69}
{'_id': 'Main Road', 'c

Name: no major issues; some are capitalised and some are numbers

###Exploring phone

In [129]:
pipe1 = {"$match": {"$and": [{"phone": {"$exists": 1}}]}}
pipe2 = {"$group":{"_id": "$phone", "count":{"$sum": 1}}}
pipe3 = {"$sort": {"count":-1}}
pipe4 = {"$project": {"name": "$phone", "count":"$count"}}
pipeline=[pipe1, pipe2, pipe3, pipe4]
test = db.osm.aggregate(pipeline)

In [130]:
for i in test:
        pprint(i)

{'_id': '08000 727282', 'count': 3}
{'_id': '+44-8721-077077', 'count': 2}
{'_id': '+442380613537', 'count': 2}
{'_id': '+44 (0)1489 796979', 'count': 2}
{'_id': '+44-1730-262746', 'count': 2}
{'_id': '+442380692603', 'count': 2}
{'_id': '01252413232', 'count': 1}
{'_id': '+441590 672148', 'count': 1}
{'_id': '01252 319232', 'count': 1}
{'_id': '+44 1730 894841', 'count': 1}
{'_id': '+44 1730 827233', 'count': 1}
{'_id': '+44 1962 854 370', 'count': 1}
{'_id': '+44 1730 893363', 'count': 1}
{'_id': '+44-1329-664843', 'count': 1}
{'_id': '+44-871-9429028', 'count': 1}
{'_id': '+44-1794-511909', 'count': 1}
{'_id': '+44-23-8052-4000', 'count': 1}
{'_id': '+44 871 527 9002', 'count': 1}
{'_id': '0800 10 72 682', 'count': 1}
{'_id': '+44 2392 464491', 'count': 1}
{'_id': '+44 2380 811212', 'count': 1}
{'_id': '+44 1420 474135', 'count': 1}
{'_id': '+44-23-8077-1286', 'count': 1}
{'_id': '+44 23 80 615 692', 'count': 1}
{'_id': '+44 238 098 7591', 'count': 1}
{'_id': '+44 23 8061 5178', 'co

Multiple issues...

### Exploring housenumber

In [138]:
pipe1 = {"$match": {"$and": [{"address.housenumber": {"$exists": 1}}]}}
pipe2 = {"$group":{"_id": "$address.housenumber", "count":{"$sum": 1}}}
pipe3 = {"$sort": {"count":-1}}
pipe4 = {"$project": {"name": "$address.housenumber", "count":"$count"}}
pipeline=[pipe1, pipe2, pipe3, pipe4]
test = db.osm.aggregate(pipeline)

In [139]:
for i in test:
        pprint(i)

{'_id': '1', 'count': 739}
{'_id': '2', 'count': 677}
{'_id': '3', 'count': 669}
{'_id': '4', 'count': 640}
{'_id': '5', 'count': 639}
{'_id': '6', 'count': 590}
{'_id': '7', 'count': 568}
{'_id': '8', 'count': 543}
{'_id': '10', 'count': 502}
{'_id': '9', 'count': 489}
{'_id': '11', 'count': 473}
{'_id': '12', 'count': 455}
{'_id': '14', 'count': 421}
{'_id': '15', 'count': 399}
{'_id': '16', 'count': 391}
{'_id': '17', 'count': 369}
{'_id': '13', 'count': 364}
{'_id': '18', 'count': 358}
{'_id': '19', 'count': 352}
{'_id': '20', 'count': 330}
{'_id': '21', 'count': 317}
{'_id': '22', 'count': 312}
{'_id': '23', 'count': 298}
{'_id': '25', 'count': 290}
{'_id': '26', 'count': 287}
{'_id': '24', 'count': 284}
{'_id': '27', 'count': 283}
{'_id': '28', 'count': 273}
{'_id': '29', 'count': 262}
{'_id': '30', 'count': 255}
{'_id': '31', 'count': 254}
{'_id': '32', 'count': 247}
{'_id': '34', 'count': 239}
{'_id': '33', 'count': 237}
{'_id': '35', 'count': 227}
{'_id': '36', 'count': 225}
{

Many inconsistencies...

###Exploring district

In [140]:
pipe1 = {"$match": {"$and": [{"address.district": {"$exists": 1}}]}}
pipe2 = {"$group":{"_id": "$address.district", "count":{"$sum": 1}}}
pipe3 = {"$sort": {"count":-1}}
pipe4 = {"$project": {"name": "$address.district", "count":"$count"}}
pipeline=[pipe1, pipe2, pipe3, pipe4]
test = db.osm.aggregate(pipeline)

In [141]:
for i in test:
        pprint(i)

{'_id': "Chandler's Ford", 'count': 6153}
{'_id': 'Ampfield', 'count': 518}
{'_id': 'Compton', 'count': 300}
{'_id': 'Shawford', 'count': 240}
{'_id': 'Chilworth', 'count': 160}
{'_id': 'Otterbourne', 'count': 97}
{'_id': 'Allbrook & North Boyatt', 'count': 13}
{'_id': 'Hursley', 'count': 6}
{'_id': 'Hampshire', 'count': 2}


Very under-completed field - Chandler's Ford is over-represented in comparison, also Hampshire should not have been a value

###Exploring amenity and cuisine

In [35]:
pipe1 = {"$match": {"$and": [{"amenity": "fast_food"}, {"cuisine": {"$exists": 0}}]}}
pipe2 = {"$group":{"_id": "$name", "count":{"$sum": 1}}}
pipe3 = {"$sort": {"count":-1}}
pipe4 = {"$project": {"name": "$name", "count":"$count"}}
pipeline=[pipe1, pipe2, pipe3, pipe4]
test = db.osm.aggregate(pipeline)
for i in test:
        pprint(i)

{'_id': None, 'count': 13}
{'_id': 'Subway', 'count': 10}
{'_id': 'KFC', 'count': 7}
{'_id': 'Wimpy', 'count': 3}
{'_id': 'Fish & Chips', 'count': 3}
{'_id': "McDonald's", 'count': 3}
{'_id': 'Greggs', 'count': 2}
{'_id': "Domino's Pizza", 'count': 2}
{'_id': 'Burger King', 'count': 2}
{'_id': 'Yellow River', 'count': 1}
{'_id': 'Express Pizza', 'count': 1}
{'_id': 'Bengal Express', 'count': 1}
{'_id': 'Ruby Takeaway', 'count': 1}
{'_id': 'Taroka Indian Take Away', 'count': 1}
{'_id': 'Chunkys', 'count': 1}
{'_id': 'Charcoal Grill', 'count': 1}
{'_id': "TC's Fish and Chips", 'count': 1}
{'_id': 'Kendal Chippy', 'count': 1}
{'_id': 'Chinese takeaway', 'count': 1}
{'_id': 'The Kiosk', 'count': 1}
{'_id': 'Oriental', 'count': 1}
{'_id': 'Harringtons', 'count': 1}
{'_id': 'Roosters', 'count': 1}
{'_id': "Tracy's Snack Bar", 'count': 1}
{'_id': 'Sweet & Sour', 'count': 1}
{'_id': "Ho's Fish Bar", 'count': 1}
{'_id': 'Tortilla', 'count': 1}
{'_id': 'Currylicious', 'count': 1}
{'_id': "O'Yes"

In [30]:
db.osm.find({"$and": [{"cuisine": {"$exists": 1}}, {"amenity": "restaurant"}]}).count()

292

In [45]:
pipe1 = {"$match": {"$and": [{"amenity": {"$exists": 1}}, {"cuisine": {"$exists": 0}}]}}
pipe2 = {"$group":{"_id": "$amenity", "count":{"$sum": 1}}}
pipe3 = {"$sort": {"count":-1}}
pipe4 = {"$project": {"name": "$amenity", "count":"$count"}}
pipeline=[pipe1, pipe2, pipe3, pipe4]
test = db.osm.aggregate(pipeline)
for i in test:
        pprint(i)

{'_id': 'parking', 'count': 3642}
{'_id': 'post_box', 'count': 2339}
{'_id': 'pub', 'count': 1088}
{'_id': 'bicycle_parking', 'count': 884}
{'_id': 'bench', 'count': 882}
{'_id': 'place_of_worship', 'count': 771}
{'_id': 'school', 'count': 752}
{'_id': 'telephone', 'count': 595}
{'_id': 'recycling', 'count': 271}
{'_id': 'cafe', 'count': 266}
{'_id': 'toilets', 'count': 260}
{'_id': 'post_office', 'count': 238}
{'_id': 'bank', 'count': 222}
{'_id': 'fuel', 'count': 220}
{'_id': 'atm', 'count': 209}
{'_id': 'pharmacy', 'count': 192}
{'_id': 'restaurant', 'count': 178}
{'_id': 'grave_yard', 'count': 134}
{'_id': 'public_building', 'count': 105}
{'_id': 'community_centre', 'count': 102}
{'_id': 'fast_food', 'count': 100}
{'_id': 'doctors', 'count': 99}
{'_id': 'grit_bin', 'count': 99}
{'_id': 'library', 'count': 71}
{'_id': 'bar', 'count': 66}
{'_id': 'shop', 'count': 63}
{'_id': 'kindergarten', 'count': 52}
{'_id': 'fire_station', 'count': 52}
{'_id': 'police', 'count': 50}
{'_id': 'dent

In [50]:
pipe1 = {"$match": {"$and": [{"$or": [{"amenity":"restaurant"}, {"amenity":"pub"},{"amenity":"cafe"},{"amenity":"fast_food"}]}, {"cuisine": {"$exists": 0}}]}}
pipe2 = {"$group":{"_id": "$amenity", "count":{"$sum": 1}}}
pipe3 = {"$sort": {"count":-1}}
pipe4 = {"$project": {"name": "$amenity", "count":"$count"}}
pipeline=[pipe1, pipe2, pipe3, pipe4]
test = db.osm.aggregate(pipeline)
for i in test:
        pprint(i)

{'_id': 'pub', 'count': 1088}
{'_id': 'cafe', 'count': 266}
{'_id': 'restaurant', 'count': 178}
{'_id': 'fast_food', 'count': 100}


### Get unique user contributors

In [13]:
pipe1 = {"$match": {"$and": [{"created.user": {"$exists": 1}}]}}
pipe2 = {"$group":{"_id": "$created.user", "count":{"$sum": 1}}}
pipe3 = {"$sort": {"count":-1}}
pipe4 = {"$project": {"name": "$created.user", "count":"$count"}}
pipeline=[pipe1, pipe2, pipe3, pipe4]
test = db.osm.aggregate(pipeline)
userc_list = []
usern_list = []
for i in test:
    userc_list.append(i["count"])
    usern_list.append(i["_id"])

Using plotly to display user contributions

In [28]:
data = Data([
    Bar(
        x=usern_list,
        y=[math.log(x) for x in userc_list]
    )
])
plot_url = py.plot(data, filename='basic-bar')

In [44]:
len(db.osm.distinct("created.user"))

1687

###Other higher level stats

In [55]:
db.osm.find({"type": "node"}).count()

1991437

In [56]:
db.osm.find({"type": "way"}).count()

261248